# Data Pre Processing and Saving

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import downscale_local_mean
from os.path import join
import cv2
from sklearn.model_selection import train_test_split

input_folder = 'Data'

df_mask = pd.read_csv(join(input_folder, 'train_masks.csv'), usecols=['img'])
ids_train = df_mask['img'].map(lambda s: s.split('_')[0]).unique()

imgs_idx = list(range(1, 17))

In [2]:
load_img = lambda im, idx: imread(join(input_folder, 'train', '{}_{:02d}.jpg'.format(im, idx)))
load_mask = lambda im, idx: imread(join(input_folder, 'train_masks', '{}_{:02d}_mask.gif'.format(im, idx)))
resize = lambda im: downscale_local_mean(im, (4,4) if im.ndim==2 else (4,4,1))
mask_image = lambda im, mask: (im * np.expand_dims(mask, 2))

## Donot Run - code for saving and preprocessing training data

In [ ]:
# num_train = len(ids_train) 
# X = np.empty((num_train*16, 320, 480, 12), dtype=np.float32)
# y = np.empty((num_train*16, 320, 480, 1), dtype=np.float32)
# for i, img_id in enumerate(ids_train):
#     imgs_id = [resize(load_img(img_id, j)) for j in imgs_idx]
#     for idx in range(1,17):
#         X[i, ..., :9] = np.concatenate([imgs_id[idx-1], np.mean(imgs_id, axis=0), np.std(imgs_id, axis=0)], \
#                                        axis=2)
#         y[i] = resize(np.expand_dims(load_mask(img_id, idx), 2)) / 255.
#     del imgs_id 

In [4]:
import bcolz

In [7]:
y=bcolz.open('y_new')
y2=y.reshape((5088,1,320,480))


In [9]:
c=bcolz.carray(y2, rootdir="y_new2", mode='w'); 
c.flush()

In [ ]:
X.shape
X2=X.reshape((5088,12,320,480))
y.shape
y2=y.reshape((5088,1,320,480))

In [ ]:
import bcolz
c=bcolz.carray(X2, rootdir="X_new", mode='w'); 
c.flush()
c=bcolz.carray(y, rootdir="y_new", mode='w'); 
c.flush()
c=bcolz.carray(X, rootdir="X_original", mode='w'); 
c.flush()
c=bcolz.carray(y, rootdir="y_original", mode='w'); 
c.flush()

# Create Valid Set -- Include Diversity

In [24]:
import random
val_set = [ (random.randint(0, len(ids_train)) ,x%16 ) for x in range(20)]


In [25]:
val_set

[(4, 0),
 (224, 1),
 (141, 2),
 (316, 3),
 (259, 4),
 (4, 5),
 (81, 6),
 (84, 7),
 (205, 8),
 (118, 9),
 (165, 10),
 (42, 11),
 (58, 12),
 (248, 13),
 (120, 14),
 (243, 15),
 (248, 0),
 (75, 1),
 (212, 2),
 (218, 3)]

In [30]:
num_val = len(val_set) 
X = np.empty((num_val, 320, 480, 9), dtype=np.float32)
y = np.empty((num_val, 320, 480, 1), dtype=np.float32)
cnt=0
for i in val_set:
    imgs_id = [resize(load_img(ids_train[i[0]], j)) for j in imgs_idx]
    idx=i[1]
    X[cnt, ..., :9] = np.concatenate([imgs_id[idx], np.mean(imgs_id, axis=0), np.std(imgs_id, axis=0)], \
                                   axis=2)
    y[cnt] = resize(np.expand_dims(load_mask(ids_train[i[0]], idx+1), 2)) / 255.
    cnt+=1
    del imgs_id 

In [31]:
c=bcolz.carray(X, rootdir="X_val", mode='w'); 
c.flush()
c=bcolz.carray(y, rootdir="y_val", mode='w'); 
c.flush()